# Term sparsity

**Adapted from**: Example 3.5 of [WML20b]

[WML20a] Wang, Jie, Victor Magron, and Jean-Bernard Lasserre.
*TSSOS: A Moment-SOS hierarchy that exploits term sparsity*.
arXiv preprint arXiv:1912.08899 (2020).

[WML20b] Wang, Jie, Victor Magron, and Jean-Bernard Lasserre.
*Chordal-TSSOS: a moment-SOS hierarchy that exploits term sparsity with chordal extension*.
arXiv preprint arXiv:2003.03210 (2020).

In [1]:
using DynamicPolynomials
@polyvar x[1:3]

(DynamicPolynomials.Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[x₁, x₂, x₃],)

We would like to find the minimum value of the polynomial

In [2]:
poly = x[1]^2 - 2x[1]*x[2] + 3x[2]^2 - 2x[1]^2*x[2] + 2x[1]^2*x[2]^2 - 2x[2]*x[3] + 6x[3]^2 + 18x[2]^2*x[3] - 54x[2]*x[3]^2 + 142x[2]^2*x[3]^2

6x₃² - 2x₂x₃ + 3x₂² - 2x₁x₂ + x₁² - 54x₂x₃² + 18x₂²x₃ - 2x₁²x₂ + 142x₂²x₃² + 2x₁²x₂²

The minimum value of the polynomial can be found to be zero.

In [3]:
import CSDP
solver = CSDP.Optimizer
using SumOfSquares
function sos_min(sparsity)
    model = Model(solver)
    @variable(model, t)
    @objective(model, Max, t)
    con_ref = @constraint(model, poly - t in SOSCone(), sparsity = sparsity)
    optimize!(model)
    return value(t), moment_matrix(con_ref)
end

bound, ν = sos_min(Sparsity.NoPattern())
bound

Success: SDP solved
Primal objective value: 0.0000000e+00 
Dual objective value: 0.0000000e+00 
Relative primal infeasibility: 1.37e-16 
Relative dual infeasibility: 5.00e-11 
Real Relative Gap: 0.00e+00 
XZ Relative Gap: 1.34e-10 
DIMACS error measures: 2.22e-16 0.00e+00 8.34e-11 0.00e+00 0.00e+00 1.34e-10
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.71e-01 Pobj: -2.4909930e+02 Ad: 8.28e-01 Dobj:  7.2430886e+02 
Iter:  2 Ap: 9.99e-01 Pobj: -1.6816542e+02 Ad: 9.32e-01 Dobj:  1.6018799e+02 
Iter:  3 Ap: 9.96e-01 Pobj: -6.7497005e+00 Ad: 9.25e-01 Dobj:  6.1375870e+01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.4717032e+00 Ad: 9.01e-01 Dobj:  8.9499894e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -5.0539294e-01 Ad: 7.90e-01 Dobj:  2.7968305e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -1.5436020e-01 Ad: 7.76e-01 Dobj:  8.7592687e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -4.8636534e-02 Ad: 8.12e-01 Dobj:  2.4142345e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -1.2922923e-02 Ad: 8.39e-01 

-3.0596397637694395e-10

We find the corresponding minimizer `(0, 0, 0)` by matching the moments
of the moment matrix with a dirac measure centered at this minimizer.

In [4]:
atomic_measure(ν, 1e-6)

Atomic measure on the variables x[1], x[2], x[3] with 1 atoms:
 at [2.0746548419820583e-6, 1.2388378355883834e-6, 2.3096229027153988e-8] with weight 1.000000001500628

We can see below that the basis contained 6 monomials hence we needed to use 6x6 PSD matrix variables.

In [5]:
ν.basis

MonomialBasis([1, x₃, x₂, x₁, x₂x₃, x₁x₂])

Using the monomial/term sparsity method of [WML20a] based on cluster completion, we find the same bound.

In [6]:
bound, ν = sos_min(Sparsity.Monomial())
bound

Iter: 14 Ap: 9.68e-01 Pobj: -3.0596398e-10 Ad: 9.70e-01 Dobj: -2.0389932e-07 
Success: SDP solved
Primal objective value: -3.0596398e-10 
Dual objective value: -2.0389932e-07 
Relative primal infeasibility: 6.74e-14 
Relative dual infeasibility: 1.56e-09 
Real Relative Gap: -2.04e-07 
XZ Relative Gap: 2.88e-09 
DIMACS error measures: 7.27e-14 0.00e+00 2.58e-09 0.00e+00 -2.04e-07 2.88e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.71e-01 Pobj: -2.4909930e+02 Ad: 8.28e-01 Dobj:  7.2430886e+02 
Iter:  2 Ap: 9.99e-01 Pobj: -1.6816542e+02 Ad: 9.32e-01 Dobj:  1.6018799e+02 
Iter:  3 Ap: 9.96e-01 Pobj: -6.7497005e+00 Ad: 9.25e-01 Dobj:  6.1375870e+01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.4717032e+00 Ad: 9.01e-01 Dobj:  8.9499894e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -5.0539294e-01 Ad: 7.90e-01 Dobj:  2.7968305e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -1.5436020e-01 Ad: 7.76e-01 Dobj:  8.7592687e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -4.8636534e-02 Ad: 8.12e

-3.0596397637694395e-10

Which is not suprising as no sparsity reduction could be performed.

In [7]:
[sub.basis for sub in ν.blocks]

1-element Vector{MonomialBasis{DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, DynamicPolynomials.MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}}}:
 MonomialBasis([1, x₃, x₂, x₁, x₂x₃, x₁x₂])

Using the monomial/term sparsity method of [WML20b] based on chordal completion, the lower bound is smaller than 0.

In [8]:
bound, ν = sos_min(Sparsity.Monomial(ChordalCompletion()))
bound

Iter: 14 Ap: 9.68e-01 Pobj: -3.0596398e-10 Ad: 9.70e-01 Dobj: -2.0389932e-07 
Success: SDP solved
Primal objective value: -3.0596398e-10 
Dual objective value: -2.0389932e-07 
Relative primal infeasibility: 6.74e-14 
Relative dual infeasibility: 1.56e-09 
Real Relative Gap: -2.04e-07 
XZ Relative Gap: 2.88e-09 
DIMACS error measures: 7.27e-14 0.00e+00 2.58e-09 0.00e+00 -2.04e-07 2.88e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.27e-01 Pobj: -1.4689079e+02 Ad: 8.89e-01 Dobj:  9.8435236e+02 
Iter:  2 Ap: 7.48e-01 Pobj: -1.1657123e+02 Ad: 8.75e-01 Dobj:  2.5250977e+02 
Iter:  3 Ap: 9.36e-01 Pobj: -3.9438731e+01 Ad: 6.27e-01 Dobj:  1.9098284e+02 
Iter:  4 Ap: 1.00e+00 Pobj: -5.2243839e+00 Ad: 8.90e-01 Dobj:  5.7737377e+01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.9667022e+00 Ad: 9.46e-01 Dobj:  9.3864604e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -6.0952393e-01 Ad: 8.98e-01 Dobj:  2.7249453e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -1.8828026e-01 Ad: 7.05e

-0.0035511999768404467

However, this bound was obtained with an SDP with 4 matrices of size 3x3.

In [9]:
[sub.basis for sub in ν.blocks]

4-element Vector{MonomialBasis{DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, DynamicPolynomials.MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}}}:
 MonomialBasis([x₂, x₁, x₁x₂])
 MonomialBasis([x₂, x₂x₃, x₁x₂])
 MonomialBasis([x₃, x₂, x₂x₃])
 MonomialBasis([1, x₂x₃, x₁x₂])

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*